<a href="https://colab.research.google.com/github/pachterlab/kb_docs/blob/main/docs/source/translated/notebooks/virus_detection_bulk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Viral proteins in bulk RNA seq data
In this tutorial, we will align RNA sequencing data collected from SARS-CoV2 infected human iPSC derived cardiomyocytes to viral RdRP protein sequences. Let's see if we can detect SARS-CoV2 RdRP-like sequences as expected. This is a SMART-seq dataset, but this workflow is the same for bulk RNA sequencing data.

References:  
RNA seq data: https://www.cell.com/cell-reports-medicine/pdf/S2666-3791(20)30068-9.pdf  
PalmDB viral protein reference database:  https://www.nature.com/articles/s41586-021-04332-2  


Written by: Laura Luebbert (last updated: 09/19/2024)

In [ ]:
# Number of threads to use during alignments
threads = 2

## Install software

In [ ]:
!pip install -q ffq gget kb_python

## Download the RNA sequencing data

In [ ]:
import json
import glob

# Get ftp download links for raw data with ffq and store results in json file
!ffq SRR11777734 SRR11777735 SRR11777736 SRR11777737 SRR11777738 SRR11777739 \
    --ftp \
    -o ffq.json

# Load ffq output
f = open("ffq.json")
data_json = json.load(f)
f.close()

# Download raw data using FTP links fetched by ffq
for dataset in data_json:
    url = dataset["url"]
    !curl -O $url

## Download the protein reference
In this case, we are using an optimized version of the [PalmDB database](https://github.com/ababaian/palmdb). The database was optimized for the detection of viral proteins from RNA sequencing data as described in [this manuscript](https://www.biorxiv.org/content/10.1101/2023.12.11.571168) and the files are stored in the [accompanying repository](https://github.com/pachterlab/LSCHWCP_2023/tree/main).

In [ ]:
# Download the ID to taxonomy mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

## Build a reference index from the viral protein sequences and mask host (here, human) sequences

In addition to building a reference index from the protein sequences, we also want to mask host (in this case, human) sequences.

Here, we are downloading a precomputed PalmDB reference index for use with `kb` in which the human genome and transcriptome were masked. You can find all available precomputed reference indeces [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/precomputed_refs).

In [ ]:
# Download precomputed index with masked human genome and transcriptome
!wget https://data.caltech.edu/records/sh33z-hrx98/files/palmdb_human_dlist_cdna_dna.idx?download=1
!mv palmdb_human_dlist_cdna_dna.idx?download=1 palmdb_human_dlist_cdna_dna.idx

**Alternatively, you can compute the PalmDB reference index yourself (and define a different host species):**

The `--aa` argument tells `kb` that this is an amino acid reference.

The `--d-list` argument is the path to the host genome/transcriptome. These sequences will be masked in the index. Here, we are using gget to fetch the human genome and transcriptome from Ensembl (release 110).

We are using `--workflow custom` here since we do not have a .gtf file for the PalmDB fasta file.

Building the index will take some time (~20 min), since the human genome is quite large.

In [ ]:
# # Build a PalmDB reference index and mask the human genome and transcriptome from scratch

# # Download the human reference genome and transcriptome
# # Replace 'homo_sapiens' here and the genome/transcriptome file names below if the host is a different species
# !gget ref -r 110 -w cdna,dna -d homo_sapiens

# # Concatenate human genome and transcriptome into one file
# !cat Homo_sapiens.GRCh38.cdna.all.fa.gz Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz > Homo_sapiens.GRCh38.cdna_dna.fa.gz

# %%time
# !kb ref \
#   --workflow custom \
#   --aa \
#   --d-list Homo_sapiens.GRCh38.cdna_dna.fa.gz \
#   -t $threads \
#   -i palmdb_human_dlist_cdna_dna.idx \
#   palmdb_rdrp_seqs.fa

## Align data using kallisto translated search

Create a batch file so we can run all fastq files simultaneously (to learn more about batch files, see Box 7 in the [Protocols paper](https://www.biorxiv.org/content/10.1101/2023.11.21.568164v2.full.pdf)):

In [ ]:
import numpy as np

# To decrease the runtime of this example tutorial,
# we will only align the top [n_seq_to_keep] sequences in each fastq file
n_seq_to_keep = 10000000

with open("batch.txt", "w") as batch_file:
  for filename in np.sort(glob.glob("*.fastq.gz")):
    # Shorten fastq file
    n_rows_to_keep = n_seq_to_keep * 4
    filename_short = filename.split(".fastq.gz")[0] + "_short.fastq"
    !zcat $filename | head -$n_rows_to_keep > $filename_short

    # Write batch.txt file
    sample_name = filename.split("/")[-1].split(".")[0]
    batch_file.write(sample_name + "\t" +  filename_short + "\n")

The `--aa` argument tells `kb` that this is an amino acid reference. The `-x` techology tells `kb` where to find the barcode and UMI in the data.

In [ ]:
%%time

!kb count \
    --aa \
    -t $threads \
    -i palmdb_human_dlist_cdna_dna.idx \
    -g palmdb_clustered_t2g.txt \
    -x bulk \
    --parity single \
    --h5ad \
    -o kb_output \
    batch.txt

## Load generated count matrix

In [ ]:
!pip install -q kb_python

import anndata
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

In [ ]:
u_tax_csv = "ID_to_taxonomy_mapping.csv"

In [ ]:
# Open count matrix (AnnData object in h5ad format)
adata = anndata.read_h5ad("kb_output/counts_unfiltered/adata.h5ad")
adata

## Plot counts for SARS-CoV in each sample

In [ ]:
# Find reference IDs for SARS-CoV proteins
tax_df = pd.read_csv(u_tax_csv)
tax_df[tax_df["species"].str.contains("Severe acute respiratory syndrome")]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))
fontsize = 16
width = 0.75

# These labels match the ordered
x_labels = ['Infected 1', 'Infected 2', 'Infected 3', 'Control 1', 'Control 2', 'Control 3']

target_ids = tax_df[tax_df["species"].str.contains("Severe acute respiratory syndrome-related coronavirus")]["rep_ID"].unique()

counts = []
samples = np.sort(adata.obs.index.values)
labels = samples
for sample in samples:
    counts.append(adata.X[adata.obs.index == sample, adata.var.index.isin(target_ids)].sum())

x = np.arange(len(labels))

ax.bar(x, counts, width=width, color="#003049", edgecolor="black")

ax.set_yscale("symlog")
ax.set_ylabel("kallisto (raw counts for SARS-CoV)", fontsize=fontsize)
# ax.set_xlabel("Sample", fontsize=fontsize)

ax.set_xticks(x, x_labels, rotation=45, ha="right")

ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"SARS-CoV-2 infected human\niPSC-derived cardiomyocytes", fontsize=fontsize+2)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

# Save figure
plt.savefig("smartseq_benchmark_PRJNA631969.png", dpi=300, bbox_inches="tight")

fig.show()

In [ ]:
counts